<a href="https://colab.research.google.com/github/PatuanPurba/JSC270_HW2_2025_PPurba/blob/main/Assignment_2_Part_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
income94 = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)
income94.columns = ['age','workclass', 'fnlwgt','education','education_num','marital_status','occupation',
                    'relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','gross_income_group']

income94 = income94.applymap(lambda x: x.strip() if isinstance(x, str) else x) # Strip the unnecessary white space

<ipython-input-2-b6a99e33d999>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  income94 = income94.applymap(lambda x: x.strip() if isinstance(x, str) else x) # Strip the unnecessary white space
